<a href="https://colab.research.google.com/github/angeruzzi/CompeticoesML/blob/main/7a_Flai_202201_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas e Fonte

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import LinearSVR

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer

import sklearn.metrics as me
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Base de Dados
fonte_treino = 'https://raw.githubusercontent.com/angeruzzi/Datasource/main/DesafioDDS202202_treino.csv'
fonte_teste = 'https://raw.githubusercontent.com/angeruzzi/Datasource/main/DesafioDDS202202_teste.csv'
orig   = pd.read_csv(fonte_treino) 
teste  = pd.read_csv(fonte_teste) 

#Funções

In [ ]:
                     # MLPRegressor(hidden_layer_sizes = (5,5), 
                     #     learning_rate='adaptive',
                     #     learning_rate_init=0.05,
                     #     solver='lbfgs',
                     #     random_state=1,
                     #     max_iter=2000),
                     # Lasso(),
                      
lista_de_modelos = [
                      #LinearRegression(),
                      #KNeighborsRegressor(),
                      #DecisionTreeRegressor(),
                      #RandomForestRegressor(),
                      #Ridge(),
                      #LassoLars(),
                      #BayesianRidge(),
                      #GradientBoostingRegressor(),
                      LGBMRegressor(),
                      LinearSVR()
                  ]

nome_dos_modelos = [
                      #'LinearReg', 
                      #'Knn',
                      #'DTree',                    
                      #'NNmlp',
                      #'RanFor',
                      #'Ridge',
                      #'Lasso',
                      #'LassoLars',
                      #'BayesianRidge',
                      #'GradBoost',
                      'LGBM',
                      'LSVR'
                    ]

In [ ]:
#Teste de Modelos
def CompareML_kfold(treinoX, treinoy, lista_de_modelos, nome_dos_modelos, validacao):  
  lista_medidas = ['r2','neg_mean_squared_error', 'neg_mean_absolute_error']
  nome_medidas = ['R2','EQM','EMA']

  resultados0 = {}
  
  for i in range(len(lista_de_modelos)):
    modelo = lista_de_modelos[i]

    testes = cross_validate(modelo, treinoX, treinoy, cv=validacao, scoring=lista_medidas)
    
    r2  = testes['test_r2'].mean()
    eqm = testes['test_neg_mean_squared_error'].mean()
    ema = testes['test_neg_mean_absolute_error'].mean()

    resultados0[nome_dos_modelos[i]] = [r2, eqm, ema]
  resultados = pd.DataFrame(resultados0, index = nome_medidas).T

  return resultados

In [ ]:
#Tunagem de Hiperparâmetros 
def Tunagem(modelo, treino, targets, parametros, validacao, score):    
    search = GridSearchCV(modelo, param_grid = parametros,
                                  scoring = score, cv = validacao, 
                                  verbose = 1, n_jobs = -1)
    search.fit(treino, targets) 
    bestModel = search.best_estimator_
    bestScore = search.best_score_
    bestParam = search.best_params_

    return {
            'bestModel': bestModel,
            'bestScore': bestScore,
            'bestParam': bestParam
    }

#Teste 1

In [ ]:
def aux_cor_raca(x):
    if x == 99:
      return 9
    else:
      return x

def aux_sexo(x):
    if x == 2:
      return 0
    else:
      return x

In [ ]:
treino1  = orig.copy()
treino1  = treino1.loc[treino1['faixa_etaria']>1].copy()

treino1X = treino1.drop(['remuneracao'], axis = 1) 
treino1y = treino1['remuneracao']

#Remoção de Campos
treino1X.drop(['id'], axis = 1, inplace = True) 
treino1X.drop(['idade'], axis = 1, inplace = True)

#Tratamentos
treino1X['cor_raca']  = treino1X['cor_raca'].apply(aux_cor_raca)
treino1X['sexo']      = treino1X['sexo'].apply(aux_sexo)

mediaTrabalho  = np.mean(treino1X['horas_trabalho_por_semana'])
desvioTrabalho = np.std(treino1X['horas_trabalho_por_semana'])
treino1X['horas_trabalho_por_semana'] = (treino1X['horas_trabalho_por_semana'] - mediaTrabalho)/desvioTrabalho

mediaTamanho  = np.mean(treino1X['tamanho_da_empresa'])
desvioTamanho = np.std(treino1X['tamanho_da_empresa'])
treino1X['tamanho_da_empresa'] = (treino1X['tamanho_da_empresa'] - mediaTamanho)/desvioTamanho

mediaTemprego  = np.mean(treino1X['tempo_no_emprego'])
desvioTemprego = np.std(treino1X['tempo_no_emprego'])
treino1X['tempo_no_emprego'] = (treino1X['tempo_no_emprego'] - mediaTemprego)/desvioTemprego


##Dummyficação
#variaveis_toDummie  = ['cor_raca'] #'faixa_etaria', 
#treino1X            = pd.get_dummies(treino1X, columns = variaveis_toDummie) #, drop_first = True

In [ ]:
treino1X.head()

,sexo,faixa_etaria,cor_raca,escolaridade,horas_trabalho_por_semana,portador_de_deficiencia,tamanho_da_empresa,tempo_no_emprego
0,1,5,8,7,0.620498,0,1.069214,-0.788159
1,1,7,8,7,0.620498,0,-1.514034,-0.807016
2,0,7,9,7,-1.179347,0,-0.037893,1.713934
3,1,4,8,7,0.620498,0,0.331143,-0.821904
4,1,7,9,9,0.106256,0,1.069214,0.493159


In [ ]:
validacao = RepeatedKFold(n_splits = 10, n_repeats = 10)
resultados = CompareML_kfold(treino1X, treino1y, lista_de_modelos, nome_dos_modelos, validacao)
resultados.sort_values(by = 'EMA', ascending = False)

,R2,EQM,EMA
LGBM,0.383546,-8.817499e+06,-1400.993181
SVR,0.127742,-1.250377e+07,-1513.407688


#Teste 2

In [44]:
treino2  = orig.copy()
treino2  = treino2.loc[treino2['faixa_etaria']>1].copy()

treino2X = treino2.drop(['remuneracao'], axis = 1) 
treino2y = treino2['remuneracao']

#Remoção de Campos
treino2X.drop(['id'], axis = 1, inplace = True) 
treino2X.drop(['idade'], axis = 1, inplace = True)

#Tratamentos
treino2X['cor_raca']  = treino2X['cor_raca'].apply(aux_cor_raca)
treino2X['sexo']  = treino2X['sexo'].apply(aux_sexo)

mediaTrabalho  = np.mean(treino2X['horas_trabalho_por_semana'])
desvioTrabalho = np.std(treino2X['horas_trabalho_por_semana'])
treino2X['horas_trabalho_por_semana'] = (treino2X['horas_trabalho_por_semana'] - mediaTrabalho)/desvioTrabalho

mediaTamanho  = np.mean(treino2X['tamanho_da_empresa'])
desvioTamanho = np.std(treino2X['tamanho_da_empresa'])
treino2X['tamanho_da_empresa'] = (treino2X['tamanho_da_empresa'] - mediaTamanho)/desvioTamanho

mediaTemprego  = np.mean(treino2X['tempo_no_emprego'])
desvioTemprego = np.std(treino2X['tempo_no_emprego'])
treino2X['tempo_no_emprego'] = (treino2X['tempo_no_emprego'] - mediaTemprego)/desvioTemprego

##Dummyficação
variaveis_toDummie  = ['cor_raca'] #,  'escolaridade', 'faixa_etaria'
treino2X            = pd.get_dummies(treino2X, columns = variaveis_toDummie) #, drop_first = True

In [ ]:
treino2X.head()

In [ ]:
validacao = RepeatedKFold(n_splits = 10, n_repeats = 10)
resultados = CompareML_kfold(treino2X, treino2y, lista_de_modelos, nome_dos_modelos, validacao)
resultados.sort_values(by = 'EMA', ascending = False)

,R2,EQM,EMA
LGBM,0.388001,-8.764554e+06,-1399.612431
SVR,0.128146,-1.250262e+07,-1512.831724


#Tunagem

In [ ]:
validacao = RepeatedKFold(n_splits = 10, n_repeats = 10)

testeTunModels = [LGBMRegressor()]
testeTunParams = [
                  {
                   'min_samples_split': [2, 3],
                   'min_samples_leaf': [1, 2],
                   'max_depth' : [5, 6, 7, 8, 9, 10],
                   'n_estimators': [50, 55, 60, 65, 70, 75],
                   'learning_rate': np.arange(0.1, 0.5, 0.1)
                  }
]

In [ ]:
dadosSelecX = treino2X
dadosSelecy = treino2y

for i in range(len(testeTunModels)):
  ret = Tunagem(testeTunModels[i], dadosSelecX, dadosSelecy, testeTunParams[i], validacao, 'neg_mean_absolute_error')
  print(ret)

Fitting 100 folds for each of 576 candidates, totalling 57600 fits
{'bestModel': LGBMRegressor(max_depth=8, min_samples_leaf=1, min_samples_split=2,
              n_estimators=65), 'bestScore': -1397.8355355052202, 'bestParam': {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 65}}


#Modelo Final e Submissão

In [45]:
fonte_saida = teste.copy()

#Remoção de Campos
fonte_saida.drop(['id'], axis = 1, inplace = True) 
fonte_saida.drop(['idade'], axis = 1, inplace = True)

#Tratamentos
fonte_saida['cor_raca']   = fonte_saida['cor_raca'].apply(aux_cor_raca)
fonte_saida['sexo']       = fonte_saida['sexo'].apply(aux_sexo)

#mediaTrabalho  = np.mean(fonte_saida['horas_trabalho_por_semana'])
#desvioTrabalho = np.std(fonte_saida['horas_trabalho_por_semana'])
fonte_saida['horas_trabalho_por_semana'] = (fonte_saida['horas_trabalho_por_semana'] - mediaTrabalho)/desvioTrabalho

#mediaTamanho  = np.mean(fonte_saida['tamanho_da_empresa'])
#desvioTamanho = np.std(fonte_saida['tamanho_da_empresa'])
fonte_saida['tamanho_da_empresa'] = (fonte_saida['tamanho_da_empresa'] - mediaTamanho)/desvioTamanho

#mediaTemprego  = np.mean(fonte_saida['tempo_no_emprego'])
#desvioTemprego = np.std(fonte_saida['tempo_no_emprego'])
fonte_saida['tempo_no_emprego'] = (fonte_saida['tempo_no_emprego'] - mediaTemprego)/desvioTemprego


##Dummyficação
variaveis_toDummie  = ['cor_raca'] 
fonte_saida         = pd.get_dummies(fonte_saida, columns = variaveis_toDummie) #, drop_first = True

In [46]:
dadosSelecionadosX = treino2X
dadosSelecionadosy = treino2y

modelo_decidido = LGBMRegressor(max_depth=8, min_samples_leaf=1, min_samples_split=2, n_estimators=65, learning_rate= 0.1)
modelo_decidido.fit(dadosSelecionadosX, dadosSelecionadosy)

LGBMRegressor(max_depth=8, min_samples_leaf=1, min_samples_split=2,
              n_estimators=65)

In [47]:
#Verificação nos dados de treino
preditos = modelo_decidido.predict(dadosSelecionadosX)
EMA = me.mean_absolute_error(dadosSelecionadosy, preditos)
EMA

1310.0867445113383

In [48]:
dadosSelecionadosX.head()

,sexo,faixa_etaria,escolaridade,horas_trabalho_por_semana,portador_de_deficiencia,tamanho_da_empresa,tempo_no_emprego,cor_raca_1,cor_raca_2,cor_raca_4,cor_raca_6,cor_raca_8,cor_raca_9
0,1,5,7,0.620498,0,1.069214,-0.788159,0,0,0,0,1,0
1,1,7,7,0.620498,0,-1.514034,-0.807016,0,0,0,0,1,0
2,0,7,7,-1.179347,0,-0.037893,1.713934,0,0,0,0,0,1
3,1,4,7,0.620498,0,0.331143,-0.821904,0,0,0,0,1,0
4,1,7,9,0.106256,0,1.069214,0.493159,0,0,0,0,0,1


In [49]:
pred = modelo_decidido.predict(fonte_saida)

subm = pd.DataFrame()
subm['remuneracao'] = pred
subm.to_csv('submissaoC.csv', index = False)

In [50]:
from google.colab import files
files.download("submissaoC.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>